In [ ]:
import pandas as pd
import numpy as np

import locale
locale.setlocale(locale.LC_ALL,'')

In [ ]:
archivo= "C:/Users/Lic.Costich/Downloads/examen/Examen Ventas 2022 Looker.xlsx"
data="Datos 2022"
listas="Listas"

dfListaClieyProv = pd.read_excel(archivo, listas, usecols="A:B").iloc[:7416]

dfListaVendedoras = pd.read_excel(archivo, listas, usecols="D:E",).iloc[:124]
dfListaVendedoras["Clave.1"] = dfListaVendedoras["Clave.1"].astype('Int64')

df = pd.read_excel(archivo, data)
df["Fecha "] = pd.to_datetime(df["Fecha "], dayfirst=True)
df["Vendedor "] = df["Vendedor "].astype('Int64')
df["Cantidad "] = df["Cantidad "] * -1
df["Costo "] = df["Costo "].mask(df["Costo "]<0,0) * -1

dataMerge = df.merge(dfListaVendedoras, how='left', left_on='Vendedor ', right_on='Clave.1').merge(dfListaClieyProv, how='left', left_on='Clave cliente', right_on='Clave')

rename_cols={'Número de movimiento ': 'movimiento_numero',
             'Documento ': 'documento_clave',
             'Clave de artículo ': 'articulo_clave',
             'Descripción ': 'articulo_descripcion',
             'Línea ': 'articulo_linea',
             'Clave cliente': 'cliente_clave',
             'Concepto ': 'documento_concepto',
             'Fecha ': 'documento_fecha',
             'Vendedor ': 'vendedora_clave',
             'Precio ': 'registro_precio',
             'Costo ': 'registro_costo',
             'Almacén ': 'almacen',
             'Tipo de documento ': 'documento_tipo',
             'Cantidad ': 'registro_cantidad',
             'Clave.1': 'vendedora_clave_2',
             'Vendedores': 'vendedora_descripcion',
             'Clave': 'cliente_clave_2',
             'Proveedores': 'cliente_descripcion'}
dataMerge.rename(columns=rename_cols, inplace=True)

In [ ]:
vendedorasCE = [21,23,6,55]
dataMerge['esVendedoraCE']= dataMerge['vendedora_clave'].isin(vendedorasCE)

dataMergeNoCE = dataMerge[dataMerge['esVendedoraCE'] == False]

usarCols = ['documento_clave',
            'documento_fecha',
            'documento_concepto',
            'almacen',
            'cliente_clave',
            'cliente_descripcion',
            'vendedora_clave',
            'vendedora_descripcion',
            'articulo_clave',
            'articulo_descripcion', 
            'registro_cantidad',
            'registro_precio',
            'registro_costo']
data = dataMergeNoCE[usarCols].reset_index(drop=True)

data['partida_precio'] = data['registro_cantidad'] * data['registro_precio']
data['partida_costo'] = data['registro_cantidad'] * data['registro_costo']
data['partida_utilidad'] = data['partida_precio'] + data['partida_costo']

In [ ]:
writer = pd.ExcelWriter("C:/Users/Lic.Costich/Downloads/examen/data.xlsx", engine="openpyxl")
data.to_excel(writer, sheet_name="data")
writer.close()
writer.handles = None

In [ ]:
def iterador_top(dimension, medida, writer):
    meses  = range(1,13)
    trimestres = [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
    num = 10 if dimension == 'cliente' else 3
    
    i = 1
    for mes in meses:
        dataMesMejor = data[data['documento_fecha'].dt.month == mes][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num]
        dataMesPeor = data[data['documento_fecha'].dt.month == mes][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=True).iloc[:num]
        dataMesMejor.to_excel(writer, sheet_name=f"mes{i}+")
        dataMesPeor.to_excel(writer, sheet_name=f"mes{i}-")
        i += 1

    i = 1
    for trimestre in trimestres:
        dataTrimestreMejor = data[data['documento_fecha'].dt.month.isin(trimestre)][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num]
        dataTrimestrePeor = data[data['documento_fecha'].dt.month.isin(trimestre)][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=True).iloc[:num]
        dataTrimestreMejor.to_excel(writer, sheet_name=f"trimestre{i}+")
        dataTrimestrePeor.to_excel(writer, sheet_name=f"trimestre{i}-")
        i += 1

    dataAñoMejor = data[[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num]
    dataAñoMejor.to_excel(writer, sheet_name=f"año+")
    dataAñoPeor = data[[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=True).iloc[:num]
    dataAñoPeor.to_excel(writer, sheet_name=f"año-")

In [ ]:
def algoritmo_top_mejor_peor(dimensiones, medidas):

    for dimension in dimensiones:
        
        for medida in medidas:

            archivo = f'top_mejor_peor_{dimension}s{medida.capitalize()}'

            writer = pd.ExcelWriter(f"C:/Users/Lic.Costich/Downloads/examen/{archivo}.xlsx", engine="openpyxl")

            iterador_top(dimension,medida, writer)

            writer.close()
            writer.handles = None


In [ ]:
dimensiones = ['vendedora', 'cliente']
medidas = ['precio','utilidad']

algoritmo_top_mejor_peor(dimensiones, medidas)

In [42]:
def iterador_80_20(dimension, medida, writer):
    meses  = range(1,13)
    trimestres = [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
    num = 10 if dimension == 'cliente' else 3
    
    i = 1
    for mes in meses:
        listVendedoras_mes = list(data[data['documento_fecha'].dt.month == mes][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num].index)

        for vendedora_mes in listVendedoras_mes:
            data_80_20_mes = data[(data['documento_fecha'].dt.month == mes)&(data[f'{dimension}_descripcion'] == vendedora_mes)][['articulo_descripcion',f'partida_{medida}']].groupby('articulo_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False)
            data_80_20_mes[f'partida_{medida}_acumulado'] = data_80_20_mes[f'partida_{medida}'].cumsum()
            data_80_20_mes[f'partida_{medida}_acum_%'] = data_80_20_mes[f'partida_{medida}_acumulado'] / data_80_20_mes[f'partida_{medida}'].sum()
            filaData_80_20_mes = int(data_80_20_mes[data_80_20_mes[f'partida_{medida}_acum_%'] < 0.8][[f'partida_{medida}_acum_%']].count().iloc[0]) + 1
            data_80_20_mes.iloc[:filaData_80_20_mes].to_excel(writer, sheet_name=f"{vendedora_mes[0:10]}_80_20_mes{i}")
        i += 1

    i = 1
    for trimestre in trimestres:
        listVendedoras_trimestre = list(data[data['documento_fecha'].dt.month.isin(trimestre)][[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num].index)

        for vendedora_trimestre in listVendedoras_trimestre:
            data_80_20_trimestre = data[(data['documento_fecha'].dt.month.isin(trimestre))&(data[f'{dimension}_descripcion'] == vendedora_trimestre)][['articulo_descripcion',f'partida_{medida}']].groupby('articulo_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False)
            data_80_20_trimestre[f'partida_{medida}_acumulado'] = data_80_20_trimestre[f'partida_{medida}'].cumsum()
            data_80_20_trimestre[f'partida_{medida}_acum_%'] = data_80_20_trimestre[f'partida_{medida}_acumulado'] / data_80_20_trimestre[f'partida_{medida}'].sum()
            filaData_80_20_trimestre = int(data_80_20_trimestre[data_80_20_trimestre[f'partida_{medida}_acum_%'] < 0.8][[f'partida_{medida}_acum_%']].count().iloc[0]) + 1
            data_80_20_trimestre.iloc[:filaData_80_20_trimestre].to_excel(writer, sheet_name=f"{vendedora_trimestre[0:10]}_80_20_trimestre{i}")
        i += 1


    listVendedoras_año = list(data[[f'{dimension}_descripcion',f'partida_{medida}']].groupby(f'{dimension}_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False).iloc[:num].index)
    
    for vendedora_año in listVendedoras_año:
        data_80_20_año = data[(data[f'{dimension}_descripcion'] == vendedora_año)][['articulo_descripcion',f'partida_{medida}']].groupby('articulo_descripcion').sum().sort_values(by=f'partida_{medida}',ascending=False)
        data_80_20_año[f'partida_{medida}_acumulado'] = data_80_20_año[f'partida_{medida}'].cumsum()
        data_80_20_año[f'partida_{medida}_acum_%'] = data_80_20_año[f'partida_{medida}_acumulado'] / data_80_20_año[f'partida_{medida}'].sum()
        filaData_80_20_año = int(data_80_20_año[data_80_20_año[f'partida_{medida}_acum_%'] < 0.8][[f'partida_{medida}_acum_%']].count().iloc[0]) + 1
        data_80_20_año.iloc[:filaData_80_20_año].to_excel(writer, sheet_name=f"{vendedora_año[0:10]}_80_20_año")

In [43]:
def algoritmo_80_20(dimensiones, medidas):

    for dimension in dimensiones:
        
        for medida in medidas:

            archivo = f'80_20_{dimension}s, artículos por {medida.capitalize()}'

            writer = pd.ExcelWriter(f"C:/Users/Lic.Costich/Downloads/examen/{archivo}.xlsx", engine="openpyxl")

            iterador_80_20(dimension,medida, writer)

            writer.close()
            writer.handles = None


In [44]:
dimensiones = ['vendedora', 'cliente']
medidas = ['precio','utilidad']

algoritmo_80_20(dimensiones, medidas)